In [36]:
import numpy as np
import pandas as pd
import string
import re
import pickle

In [37]:
vocab = pd.read_csv('../static/model/vocabulary.txt', header=None)
tokens= vocab[0].tolist()

In [38]:
with open('../static/model/model.pkl', 'rb') as f:
    model = pickle.load(f)

In [39]:
with open('../static/model/corpora/stopwords/english','r') as file:
    sw = file.read().splitlines()

In [40]:
def remove_punctuation(text):
    for  punctuation in string.punctuation:
        text = text.replace(punctuation,'')
    return text



In [41]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [42]:

def preprocessing(text):
    data = pd.DataFrame([text], columns=["tweet"])
    data["tweet"] = data["tweet"].apply(lambda x: x.lower())
    data["tweet"]=data["tweet"].apply(lambda x:" ".join(x.lower()for x in x.split()))
    data["tweet"].apply(lambda x:" ".join(re.sub(r'^https?:\/\/.*[\r\n]*','',x,flags=re.MULTILINE)for x in x.split()))
    data["tweet"]=data["tweet"].apply(remove_punctuation)
    data["tweet"] = data['tweet'].str.replace(r'\d+', '', regex=True)
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(x for x in x.split()  if x not in sw))
    data["tweet"] = data["tweet"].apply(lambda x:" ".join(ps.stem(x) for x in x.split()))

    return data["tweet"]


In [43]:
def vectorizer(ds,vocabulary):
    vectorized_lst=[]

    for sentence in ds:
        sentence_lst = np.zeros(len(vocabulary))

        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_lst[i]=1

        vectorized_lst.append(sentence_lst)

    vectorizet_lst_new = np.asarray(vectorized_lst,dtype=np.float32)

    return vectorizet_lst_new

In [53]:
def get_prediction(vectorized_text):
    prediction = model.predict(vectorized_text)
    if prediction == 1:
        return "Positive"
    else:
        return "Negative"

In [56]:
txt1 = "That bad , Not awesome."
preprocessing_txt =preprocessing(txt1)
vectorized_txt = vectorizer(preprocessing_txt, tokens)
prediction = get_prediction(vectorized_txt)
prediction

'Negative'